In [60]:
import re
import tarfile
import functools

import numpy as np
import spacy 


from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

nlp = spacy.load('en')


In [61]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return ([x.txt.strip()  for x in nlp(sent)], [x.pos for x in nlp(sent)])

In [62]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true,
    only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [63]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: functools.reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) 
            for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [64]:
def vectorize_stories(data, word_idx, pos_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    ys = []
    xspos = []
    xqspos = []
    for story, query, answer in data:
        #tuple is created where the index 0 is the sentence- list of words
        #index 1 is essentially list of pos values
        x = [word_idx[w] for w in story[0]]
        xq = [word_idx[w]for w in query[0]]
        xpos = story[1]
        xqpos = query[1]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xqs.append(xq)
        ys.append(y)
        xqspos.append(xqpos)
        xspos.append(xpos)
    #print[w[0]]
        #xqs = [word.pos[w] for w in answer]
    return (pad_sequences(xs, maxlen=story_maxlen), pad_sequences(xqs, maxlen=query_maxlen),
           pad_sequences(xspos, maxlen=story_maxlen), pad_sequences(xqspos, maxlen=query_maxlen),np.array(ys))

In [65]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 40
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


In [66]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

In [67]:
# Default QA1 with 1000 samples
challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
# QA2 with 1000 samples
#challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
train = get_stories(tar.extractfile(challenge.format('train')))
test = get_stories(tar.extractfile(challenge.format('test')))

AttributeError: 'spacy.tokens.token.Token' object has no attribute 'txt'

In [55]:
vocab = set()
for story, q, answer in train + test:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

In [56]:
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
story_maxlen = max(map(len, (x for x, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, x, _ in train + test)))

In [57]:
x, xq, xqpos, xpos, y = vectorize_stories(train, word_idx, story_maxlen, query_maxlen)
tx, txqpos, txpos, txq, ty = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)

TypeError: vectorize_stories() missing 1 required positional argument: 'query_maxlen'

In [68]:
print('vocab = {}'.format(vocab))
print('x.shape = {}'.format(x.shape))
print('xq.shape = {}'.format(xq.shape))
print('y.shape = {}'.format(y.shape))
#print ('xqpos.shape = {}'.format(xqpos.shape))
#print ('xpos.shape = {}'.format(xpos.shape))
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))

vocab = ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'apple', 'back', 'bathroom', 'bedroom', 'discarded', 'down', 'dropped', 'football', 'garden', 'got', 'grabbed', 'hallway', 'is', 'journeyed', 'kitchen', 'left', 'milk', 'moved', 'office', 'picked', 'put', 'the', 'there', 'to', 'took', 'travelled', 'up', 'went']
x.shape = (1000, 552)
xq.shape = (1000, 5)
y.shape = (1000, 36)
story_maxlen, query_maxlen = 552, 5


## Individual Word-Embeddings

In [23]:
print('Build model...')

print('Build model...')
vectorize_stories(data, word_idx, pos_idx, story_maxlen, query_maxlen)

sentence = layers.Input(shape=(story_maxlen,), dtype='int32') # Generate input layer for vector
                                                              # of word indexes for story
encoded_sentence = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence) # Embeds vocab indexes to separate
                                                                             # n-dimensional vector representation
encoded_sentence = layers.Dropout(0.3)(encoded_sentence) # Perform dropout


question = layers.Input(shape=(query_maxlen,), dtype='int32') # Input layer for vector of word indexes for question
encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question) # Embeds indexes to n-dimensional vector
encoded_question = layers.Dropout(0.3)(encoded_question) # Perform Dropout
encoded_question = RNN(EMBED_HIDDEN_SIZE)(encoded_question) # Create LSTM layer to ~*embed the question into 50-dim*~
encoded_question = layers.RepeatVector(story_maxlen)(encoded_question)  # Repeats question embedding for output

merged = layers.concatenate([encoded_sentence, encoded_question]) # Stacks each story word with the question embedding
merged = RNN(EMBED_HIDDEN_SIZE)(merged) # Feed in ([100-dim words + question] x 66 sentence length) into a 50-dim output
merged = layers.Dropout(0.3)(merged) # Perform dropout
preds = layers.Dense(vocab_size, activation='softmax')(merged) # Add softmax output for vocab

model = Model([sentence, question], preds)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Build model...
Build model...


In [24]:
print('Training')
model.fit([x, xq], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
loss, acc = model.evaluate([tx, txq], ty,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

Training
Train on 950 samples, validate on 50 samples
Epoch 1/40
320/950 [=========>....................] - ETA: 25s - loss: 3.5379 - acc: 0.1437

KeyboardInterrupt: 

## Story-Sentence Embedding

In [25]:
print('Build model...')

sentence = layers.Input(shape=(story_maxlen,), dtype='int32') # Generate input layer for vector
                                                              # of word indexes for story
encoded_sentence = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence) # Embeds vocab indexes to separate
                                                                             # n-dimensional vector representation
encoded_sentence = layers.Dropout(0.3)(encoded_sentence) # Perform dropout
encoded_sentence = RNN(EMBED_HIDDEN_SIZE)(encoded_sentence) # Create LSTM layer to ~*embed the story into 50-dim*~

question = layers.Input(shape=(query_maxlen,), dtype='int32') # Input layer for vector of word indexes for question
encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question) # Embeds indexes to n-dimensional vector
encoded_question = layers.Dropout(0.3)(encoded_question) # Perform Dropout
encoded_question = RNN(EMBED_HIDDEN_SIZE)(encoded_question) # Create LSTM layer to ~*embed the question into 50-dim*~

merged = layers.concatenate([encoded_sentence, encoded_question]) # Stacks the story embedding with the question embedding
merged = layers.Dropout(0.3)(merged) # Perform dropout
preds = layers.Dense(vocab_size, activation='softmax')(merged) # Add softmax output for vocab

model = Model([sentence, question], preds)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Build model...


In [26]:
print('Training')
model.fit([x, xq], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
loss, acc = model.evaluate([tx, txq], ty,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

Training
Train on 950 samples, validate on 50 samples
Epoch 1/40


KeyboardInterrupt: 